# Converting traces in raw format

In this prerequisite part, we will first change the trace format, from the npy generated by the capture, to a "raw" format, simply consisting of writing the different values measured consecutively. Of course, the number of samples in one trace and the number of traces must be kept alongside the file (for example in a header file with #define).
First, let us make a capture.

In [198]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'

In [199]:
%run "../Helper_Scripts/Setup_Generic.ipynb"

In [200]:
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)

In [201]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes.s aes-independant.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes.d aes-independant.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes.i aes-independant.i
.
-------- begin --------
avr-gcc (GCC) 4.9.2
Copyright (C) 2014 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpleserial-aes.c
avr-gcc -c -mmcu=atxme

In [202]:
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3985 bytes


In [6]:
# Capture Trace
import time

ktp = cw.ktp.Basic()

key, text = ktp.next()
traces = []

nb_samples = 3000
scope.adc.samples = nb_samples
scope.adc.offset = 0

trace = cw.capture_trace(scope, target, text, key)
traces.append(trace)

We will now print the first 10 values and write the trace to a file in "raw" format.

In [7]:
import numpy as np
import os

traces_dir = 'traces'
if not os.path.exists(traces_dir) :
    os.mkdir(traces_dir)
trace_array = np.asarray([trace.wave for trace in traces])
for i in range(10):
    print('%f' %trace_array[0][i])

# Write trace to file
f = open(os.path.join(traces_dir, 'traces.raw'), "w")
data = trace_array.astype('float64')
trace_array.astype('float').tofile(os.path.join(traces_dir, 'traces.raw'))

for i in range(10):
    print('%f' %trace_array[0][i])

0.072266
-0.314453
-0.143555
-0.146484
0.013672
-0.413086
-0.215820
-0.201172
-0.026367
-0.475586
0.072266
-0.314453
-0.143555
-0.146484
0.013672
-0.413086
-0.215820
-0.201172
-0.026367
-0.475586


You must now write a C program which reads the file into an array and prints the first 10 values, using the following code skeleton, in which path is a buffer containing the file name.

    #define NB_SAMPLES 3000  // should be in a header file !
    
    double traces[NB_SAMPLES];  //only one trace here, to be adapted if needed
    
    if ((fd = open(path, O_RDONLY)) == -1) {
        perror("open");
        return -1;
    }

    if (read(fd, traces, sizeof(double) * NB_SAMPLES) == -1) {
        perror("read");
        return -1;
    }
    
    for (int i = 0; i < 10; i += 1) {
        printf("%f\n", traces[i]);
    }
    

In [8]:
%%file /home/vagrant/work/projects/chipwhisperer/TME_SCA/parser.c
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

#define NB_SAMPLES 3000 // should be in a header file !

const char *path = "traces/traces.raw";

double traces[NB_SAMPLES]; // only one trace here, to be adapted if needed

int main(int argc, char const *argv[]) {
  int fd;
  if ((fd = open(path, O_RDONLY)) == -1) {
    perror("open");
    return -1;
  }

  if (read(fd, traces, sizeof(double) * NB_SAMPLES) == -1) {
    perror("read");
    return -1;
  }

  for (int i = 0; i < 10; i += 1) {
    printf("%f\n", traces[i]);
  }
  return 0;
}

Overwriting /home/vagrant/work/projects/chipwhisperer/TME_SCA/parser.c


In [9]:
!gcc /home/vagrant/work/projects/chipwhisperer/TME_SCA/parser.c -o /home/vagrant/work/projects/chipwhisperer/TME_SCA/parser

In [10]:
!/home/vagrant/work/projects/chipwhisperer/TME_SCA/parser

0.072266
-0.314453
-0.143555
-0.146484
0.013672
-0.413086
-0.215820
-0.201172
-0.026367
-0.475586


Verify that you observe the same 10 values than in python. Change the python code above in order to convert the paintext and the key in raw type, using `astype('uint8')`, and write them in separate files.
You can check the content of the raw key file (for example) using either the linux commands `xxd` or `hexdump`.

Modify your script in order to generate raw files for a set of several traces with different random plaintexts. The file containing the traces and the file contaning the plaintext must have a length of `nb_samples * nb_traces * sizeof(double)` and `16 * nb_traces` respectively, which you can check with the linux `ls -l` command.

Note: it is possible that some traces contain an incorrect number of samples. It is advised to check the length of the trace right after its capture, and redo the capture if it is incorrect.

# Implementing CPA

You must now implement a CPA (Correlation Power Analysis) in C, using traces that you have converted as above. It should be much faster to execute in C than in python. You must use the Pearson correlation coefficient as presented during the lectures. 

Your code must be as generic as possible regarding the number of samples, traces, and file names. It should allow to perform the attack on a sub-interval of the traces, by reading into memory only this sub-interval for each trace.

In your report, you must present the results of an evaluation regarding the effect of the trace number on the success rate of the attack (in terms of number of key bytes correctly recovered), and compare it with the DPA results.

Finally, as a bonus, you can implement the DPA attack in C in order to compare the execution time between both attacks, and between the C and the python implementation. Explain the results that you have obtained in your report.

### Writing the C header file

In [3]:
%%file /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/cpa/cpa_attack.h
#ifndef CPA_ATTACK_H
# define CPA_ATTACK_H

#include <stdio.h>
#include <stdlib.h>
#include <strings.h>
#include <math.h>
#include <stdint.h>

#define KEY_SIZE 16
#define WAVE_SIZE 3000

/*
We have 2 different ways to perform the CPA Attack : 
  - Use the whole wave
  - Use only leakage ranges of the wave
Because of that we opted for a union type, so that we can either have :
  - The whole wave as a double array
  - A range for each SBOX output, which result in 16 arrays of type double
*/

typedef union         
{
  double* all;
  double* intervals[KEY_SIZE];
} u_wave ;

// Trace structure in C
typedef struct s_trace
{
  u_wave     wave;   
  int        textin[KEY_SIZE];
  int        key[KEY_SIZE];

}  t_trace;

// Precalculated Hamming Weight array: HW = [  i.count('1') for i in range(256)]
double HW[256] = {
    0.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 3.0, 1.0, 2.0, 2.0, 3.0, 2.0, 3.0, 3.0, 4.0, 1.0, 2.0, 2.0, 
    3.0, 2.0, 3.0, 3.0, 4.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 1.0, 2.0, 2.0, 3.0, 2.0, 3.0, 
    3.0, 4.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 
    4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 1.0, 2.0, 2.0, 3.0, 2.0, 3.0, 3.0, 4.0, 2.0, 3.0, 3.0, 4.0, 
    3.0, 4.0, 4.0, 5.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 
    6.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 3.0, 4.0, 
    4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 6.0, 5.0, 6.0, 6.0, 7.0, 1.0, 2.0, 2.0, 3.0, 2.0, 
    3.0, 3.0, 4.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 
    3.0, 4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 4.0, 
    5.0, 4.0, 5.0, 5.0, 6.0, 3.0, 4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 6.0, 5.0, 6.0, 
    6.0, 7.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 3.0, 
    4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 6.0, 5.0, 6.0, 6.0, 7.0, 3.0, 4.0, 4.0, 5.0, 
    4.0, 5.0, 5.0, 6.0, 4.0, 5.0, 5.0, 6.0, 5.0, 6.0, 6.0, 7.0, 4.0, 5.0, 5.0, 6.0, 5.0, 6.0, 6.0, 
    7.0, 5.0, 6.0, 6.0, 7.0, 6.0, 7.0, 7.0, 8.0
};

int dpa_leakage_points[KEY_SIZE ]= {  1505, 1760, 2010, 2262,\
                                      1563, 1814, 2067, 2321,\
                                      1618, 1871, 2124, 2376,\
                                      1676, 1927, 2180, 2429 
                                    };

// Predefined AES SBOX
uint8_t sbox[256] =   {
  //0     1    2      3     4    5     6     7      8    9     A      B    C     D     E     F
  0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
  0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
  0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
  0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
  0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
  0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
  0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
  0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
  0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
  0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
  0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
  0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
  0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
  0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
  0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
  0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16

};
//------------------------------Trace related functions-----------------------------------------------------
void print_array(int *a);                                               // prints an array of size KET_SIZE
void print_samples(double **s, int i[]);                                // prints a wave from trace
void print_traces(t_trace *t, int n, int i[], int all);                 // prints the whole trace  
u_wave init_intervals(int* i);                                          // allocates the wave intervals
u_wave init_wave(int n);                                                // allocates the a wave of size nb_samples
void fill_interval(u_wave w, int i, double* v, int s);                  // fills the wave intervals
void fill_wave(u_wave w, double* v, int n);                             // fills the whole wave
void free_intervals(u_wave w);                                          // frees the wave intervals
void free_wave(u_wave w);                                               // frees the whole wave

//------------------------------HW related functions-----------------------------------------------------
unsigned int intermediate(int pt, int keyguess);                        // calculates sbox output

//------------------------------Formula related functions------------------------------------------------
void print_d_array(double* a, int len);                                 // prints a double array of size len
void init_array(double* a, int len, double v);                          // initializes an array to a specific value
double max_array(double* a,  int len);                                  // returns the max of an array
int argmax_array(double* a,  int len);                                  // return sthe max's index of an array
void sub_arrays(double* a, double* b, double* c, int len);              // substractes 2 arrays
void add_arrays(double* a, double* b, double* c, int len);              // adds 2 arrays
void mul_arrays(double* a, double* b, double* c, int len);              // multiplies 2 arrays 
void div_arrays(double* a, double* b, double* c, int len);              // divises 2 arrays
void sqrt_array(double* a, double* b, int len);                         // calculates the square root of all the array values
double mean(double *array, int  len);                                   // calulates the mean of an array                                                     
void mean_columns(t_trace *t, double* a, int b, int c, int d, int all); // calculates the mean of columns for a 2D array

//------------------------------CPA Attack related functions------------------------------------------------------------------
double corr_coef(t_trace *t, int n, int g, int p, int b, int all);      // calculates the correlation coef of a guess
int cpa_attack_byte(t_trace *t, int n, int b, int p, int all);          // recover a subkey using CPA attack 
void cpa_attack(t_trace *tr, int n, int* i, int all);                   // recover the AES key using CPA attack
int dpa_attack_byte(t_trace *tr, int n, int b);                          // recover a subkey using DPA attack
void dpa_attack(t_trace *tr, int n);                                    // recover the AES key using DPA attack
#endif


Overwriting /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/cpa/cpa_attack.h


### Writing the CPA Attack code in C 

In [37]:
%%file /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/cpa/cpa_attack.c
#include "cpa_attack.h"

//-----------------Trace related functions-----------------------------------

void print_array(int *array)                                // prints an array of size KET_SIZE                                       
{
    for(int i = 0; i < KEY_SIZE; i++)
        printf("%d ", array[i]);
    printf("\n");
}
void print_samples(double **samples, int intervals[])       // prints a wave from trace
{
     int nr_samples = 0;
     for (int i = 0; i < KEY_SIZE; i++)
     {
           nr_samples = intervals[i];
           printf("\tintervals[%d]: ",i);
           for(int j = 0; j < nr_samples; j++)
                printf("%.10f ", samples[i][j]);
           printf("\n");
     }
}

void print_traces(t_trace *tr, int nr_traces, int intervals[], int all) // prints the whole trace
{
    for(int i = 0; i < nr_traces; i++)
    {
        printf("Trace %d :\n", i);

        if (all == 1)
        {
            printf("Wave: "); 
            print_d_array(tr[i].wave.all, WAVE_SIZE);
        }
        else 
        {
            print_samples(tr[i].wave.intervals, intervals);
        }
            
        printf("\n");
        printf("\ttextin: "); print_array(tr[i].textin);
        printf("\tkey: "); print_array(tr[i].key);
    }
}

u_wave init_intervals(int* intervals)                                 // allocates the wave intervals
{
    u_wave wave;
    for(int i = 0; i < KEY_SIZE; i++)
        wave.intervals[i] = malloc(sizeof(double) * intervals[i]);
    
    return wave;
}

u_wave init_wave(int nb_samples)                                 // allocates the a wave of size nb_samples
{
    u_wave wave;

    wave.all = malloc(sizeof(double) * nb_samples);
    
    return wave;
}

void fill_intervals(u_wave wave, int index, double* values, int size) // fills the wave intervals
{
    for(int i = 0; i < size; i++)
        wave.intervals[index][i] = values[i];
}

void fill_wave(u_wave wave, double* values, int nb_samples)          // fills the whole wave
{
    for(int i = 0; i < nb_samples; i++)
        wave.all[i] = values[i];
}

void free_intervals(u_wave wave)                                     // frees the wave intervals
{
    for(int i = 0; i < KEY_SIZE; i++)
        free(wave.intervals[i]);
}

void free_wave(u_wave wave)                                     // frees the whole wave
{
    free(wave.all);
}

//------------------------------HW related functions-----------------------------------------------------

unsigned int intermediate(int pt, int keyguess)                 // calculates sbox output
{
    return sbox[pt ^ keyguess];
}
//------------------------------Formula related functions------------------------------------------------

void print_d_array(double* a, int len)                          // prints a double array of size len
{
    for (int i = 0; i < len; i++)
        printf("%.8f ", a[i]);
    printf("\n");
}

void init_array(double* a, int len, double v) {                 // initializes an array to a specific value
    for (int i = 0; i < len; i++)
        a[i] = v;
}

double max_array(double* a, int len)  {                        // returns the max of an array
    double max = 0;
    for (int i = 0; i < len; i++) 
        if (fabs(a[i]) > max)
            max = fabs(a[i]);   
    return max;
}

int argmax_array(double* a, int len)  {                        // return sthe max's index of an array
    double max = 0.0;
    int index_max = -1;
    for (int i = 0; i < len; i++) 
        if (a[i] > max) 
        {   
            max = a[i];
            index_max = i;
        }   
    return index_max;
}

void sub_arrays(double* a, double* b, double* c, int len)  {    // substractes 2 arrays
    for (int i = 0; i < len; i++) 
        c[i] = (a[i] - b[i]);
}

void add_arrays(double* a, double* b, double* c, int len)  {    // adds 2 arrays
    for (int i = 0; i < len; i++) 
        c[i] = (a[i] + b[i]);
}

void mul_arrays(double* a, double* b, double* c, int len)  {    // multiplies 2 arrays 
    for (int i = 0; i < len; i++) 
        c[i] = (a[i] * b[i]);
}

void div_arrays(double* a, double* b, double* c, int len)  {    // divises 2 arrays
    for (int i = 0; i < len; i++) 
        c[i] = (a[i] / b[i]);
}

void sqrt_array(double* a, double* b, int len)  {               // calculates the square root of all the array values
    for (int i = 0; i < len; i++) 
        b[i] = sqrt(a[i]);
}

double mean(double* array, int len) {                          // calulates the mean of an array          
    double result = 0;
    for (int i = 0; i < len; i++)
        result += array[i];
    return result / len;
}

void mean_columns(t_trace *traces, double* mean_columns, int nb_traces, int nb_samples, int bnum, int all)  {    // calculates the mean of columns for a 2D array
    for (int i = 0; i < nb_samples; i++) {
        double result = 0.0;
        for (int j = 0; j < nb_traces; j++) {
            if (all == 1)
                result += traces[j].wave.all[i];
            else 
                result += traces[j].wave.intervals[bnum][i];
        }
        mean_columns[i] = result / nb_traces;
    }
}

//------------------------------CPA Attack related functions------------------------------------------------------------------
double corr_coef(t_trace *traces, int nb_traces, int guess, int nb_samples, int bnum, int all) { // calculates the correlation coef of a guess
    
    double COV_X_Y[nb_samples]; init_array(COV_X_Y, nb_samples, 0.0);                   // start initializing vars
    double STD_DEV_X[nb_samples];   init_array(STD_DEV_X,   nb_samples, 0.0);
    double STD_DEV_Y[nb_samples];   init_array(STD_DEV_Y,   nb_samples, 0.0);
    
    double hypothesis_mean;
    double hypothesis[nb_traces];        init_array(hypothesis, nb_traces, 0.0);
    double traces_col_means[nb_samples]; init_array(traces_col_means, nb_samples, 0.0);

    double hypothesis_diff;
    double traces_col_diff[nb_samples];  init_array(traces_col_diff, nb_samples, 0.0);
    double result[nb_samples];           init_array(result, nb_samples, 0.0);           // end initializing vars

    for (int tnum=0; tnum<nb_traces; tnum++) 
        hypothesis[tnum] = HW[ intermediate(traces[tnum].textin[bnum], guess) ];        // defining our hypothesis : for all traces, calculate the HW using 'guess' and 'traces[i].plaintext[bnum]'
    
    hypothesis_mean = mean(hypothesis, nb_traces);                                      // calculating the mean of our hypothesis
    mean_columns(traces, traces_col_means, nb_traces, nb_samples, bnum, all);

    for (int tnum=0; tnum<nb_traces; tnum++) {                                          // https://eprint.iacr.org/2015/260.pdf  (4- Incremental Pearson)
        hypothesis_diff = (hypothesis[tnum] - hypothesis_mean);
        if (all == 1 )
            sub_arrays( traces[tnum].wave.all,  traces_col_means, traces_col_diff, nb_samples ); // ∑ni=1 yi 
        else 
            sub_arrays( traces[tnum].wave.intervals[bnum],  traces_col_means, traces_col_diff, nb_samples ); // ∑ni=1 yi 

        init_array(result, nb_samples, hypothesis_diff);                                             // ∑ni=1 xi
        mul_arrays(traces_col_diff, result, result, nb_samples);                                     // ∑ni=1 xi . ∑ni=1 yi
        add_arrays(COV_X_Y, result, COV_X_Y, nb_samples);                                            // n . ∑ni=1 xi . yi - ∑ni=1 xi . ∑ni=1 yi  = COV(X,Y)
             
        init_array(result, nb_samples, hypothesis_diff * hypothesis_diff );                          // ∑ni=1 xi^2 
        add_arrays(STD_DEV_X, result, STD_DEV_X, nb_samples);                                        // n . ∑ni=1 xi^2 − ( ∑ni=1 xi )^2     = Deviation(X)  


        mul_arrays(traces_col_diff, traces_col_diff, result, nb_samples);
        add_arrays(STD_DEV_Y, result, STD_DEV_Y, nb_samples);                                        // n . ∑ni=1 yi^2− ( ∑ni=1 yi )^2      = Deviation(Y)                   
    }

    mul_arrays(STD_DEV_X, STD_DEV_Y, result, nb_samples);                                            // Deviation(Y) . Deviation(Y)                                      
    sqrt_array(result, result, nb_samples);                                                          // SQRT( Deviation(Y) . Deviation(Y) ) 
    div_arrays(COV_X_Y, result, result, nb_samples ) ;                                               // COV(X,Y) /  SQRT( Deviation(Y) . Deviation(Y) ) 

    return max_array(result, nb_samples);
}

int cpa_attack_byte(t_trace *tr, int nb_traces, int bnum, int nb_samples, int all) {       // recover a subkey using cpa attack 
    
    double corr_array[256];
    int best_guess = -1;

    for (int guess = 0; guess < 256; guess++) 
        corr_array[guess] = corr_coef(tr, nb_traces, guess, nb_samples, bnum, all);      // for each guess calculate the best correlation

    best_guess = argmax_array(corr_array, 256);                                          // the correct subkey is the guess that has the maximum correlation
    return best_guess;
}

void cpa_attack(t_trace *tr, int nb_traces, int* intervals, int all) {                    // recover all the subkeys using cpa attack
    int recovered_key[KEY_SIZE];

    for (int bnum = 0; bnum < KEY_SIZE; bnum++)                                           // for each subkey 
        recovered_key[bnum] = cpa_attack_byte(tr, nb_traces, bnum, intervals[bnum], all); // apply cpa
    
    printf("Recovered key: ");
    print_array(recovered_key);
}

int dpa_attack_byte(t_trace *tr, int nb_traces, int bnum)
{

    double mean_diffs[256];  init_array(mean_diffs, 256, 0.0);
    double threshold = 4.0;

    double group1[nb_traces]; int g1_index; double g1_avg = 0;
    double group2[nb_traces]; int g2_index; double g2_avg = 0; 
    
    int avg_point = -1;
    double hw = 0;
    double maximum = 0;
    int best_guess = -1;

    for (int guess = 0; guess < 256; guess++) 
    {
       
        init_array(group1, nb_traces, 0.0);
        init_array(group2, nb_traces, 0.0);
        g1_index = 0;
        g2_index = 0;

        for (int tnum = 0; tnum < nb_traces; tnum++ )
        {
            
            avg_point = dpa_leakage_points[bnum];
            hw = HW[ intermediate(tr[tnum].textin[bnum], guess) ]; 
            
            if ( hw < threshold ) 
                group1[g1_index++] = tr[tnum].wave.all[avg_point];
            else 
                group2[g2_index++] = tr[tnum].wave.all[avg_point];
        }
        
        g1_avg = mean(group1, g1_index);
        g2_avg = mean(group2, g2_index);
        
        mean_diffs[guess] = fabs(g1_avg-g2_avg);
        if (mean_diffs[guess] > maximum ) {
            maximum = mean_diffs[guess];
            best_guess = guess;
        }      
                
    }
    
    return best_guess;
}

void dpa_attack(t_trace *tr, int nb_traces)
{     
    int recovered_key[KEY_SIZE];     

    for (int bnum = 0; bnum < KEY_SIZE; bnum++)
        recovered_key[bnum] =  dpa_attack_byte(tr, nb_traces, bnum);
    
    printf("Recovered key: ");
    print_array(recovered_key);
}

Overwriting /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/cpa/cpa_attack.c


### Writing a Makefile

In [5]:
%%file /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/Makefile
CC = gcc  # C compiler
CFLAGS = -fPIC -Wall -Wextra -Werror # C flags
LDFLAGS = -shared   # linking flags
RM = rm -f   # rm command
TARGET_LIB = cpa/cpa_attack.so  # target lib

SRCS = cpa/cpa_attack.c  # source files

OBJS = $(SRCS:.c=.o)

.PHONY: all
all: ${TARGET_LIB}

$(TARGET_LIB): $(OBJS)
	$(CC) ${LDFLAGS} -o $@ $^
	@echo "\033[1;32m ==========[ CPA ATTACK LIBRARY SUCCESSFULLY CREATED ]=========="


.PHONY: clean
clean:
	-${RM} ${TARGET_LIB} ${OBJS}
	@echo "\033[1;31m$(TARGET_LIB) ==> DELETED"
	@echo "\033[1;31m$(OBJS) ==> DELETED"

Overwriting /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/Makefile


In [38]:
! make clean

rm -f    cpa/cpa_attack.so   cpa/cpa_attack.o
cpa/cpa_attack.so   ==> DELETED
cpa/cpa_attack.o ==> DELETED


In [39]:
! make all

gcc   -fPIC -Wall -Wextra -Werror    -c -o cpa/cpa_attack.o cpa/cpa_attack.c
gcc   -shared    -o cpa/cpa_attack.so cpa/cpa_attack.o
 ==========[ CPA ATTACK LIBRARY SUCCESSFULLY CREATED ]==========


### Definining the database Model for the Trace

In [16]:
%%file /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/database_utils.py
from peewee import Model, CharField, BlobField, SqliteDatabase, DatabaseProxy
import struct

KEY_LEN = 16

database = DatabaseProxy()

class Trace(Model):
    key = CharField(max_length=KEY_LEN)
    textin = CharField(max_length=KEY_LEN)
    wave = BlobField()

    class Meta:
        database = database

class Database:
    def __init__(self, file='traces.db'):
        self.db = SqliteDatabase(file)
        database.initialize(self.db)

    def fill_db(self, traces, nb_samples):
        self.db.create_tables([Trace])
        for wave, textin, _, key in traces:
            trace = Trace.create(
                    key    = key.hex(),
                    textin = textin.hex(),
                    wave   = struct.pack('d' * nb_samples, *wave))
            trace.save()

Overwriting /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/database_utils.py


### CAPTURE_NEW_TRACES_CELL

In [225]:
from tqdm import tnrange
from database_utils import Database, Trace, KEY_LEN
import os

def capture_traces(nb_samples, nb_traces, fixed_key) :
    scope.adc.samples = nb_samples
    scope.adc.offset = 0

    ktp = cw.ktp.Basic() # object dedicated to the generation of key (fixed by default) and plain text

    traces = [] # list of traces
    N = nb_traces  # Number of traces
    
    for i in tnrange(N, desc = 'Capturing traces'):
        key , text = ktp.next()  # creation of a pair comprising (fixed) key and text 
        trace = cw.capture_trace(scope, target, text, fixed_key) # a trace is composed of the following fields :
                                                           #    a wave (samples)
                                                           #    textin (input text), textout (output text)
                                                           #    key (input key)    
        if trace is None:
            continue
        traces.append(trace)
    return traces

def create_db(new_capture, db_file='traces.db'):
    
    if os.path.exists(db_file) :
        os.remove(db_file)
    
    db  = Database(file=db_file)
    if new_capture:
        # capture traces for fixed key
        fixed_key = bytearray.fromhex(os.urandom(16).hex())
        print("[*] Capturing new traces with a fixed key: {}".format(list(fixed_key)))
        traces  = capture_traces(nb_samples, nb_traces, fixed_key)
        db.fill_db(traces, nb_samples)

nb_traces = 50
nb_samples  = 3000
new_capture = False
db_file     = "traces.db"
create_db(new_capture,db_file)

[*] Capturing new traces with a fixed key: [64, 21, 79, 190, 114, 152, 34, 43, 114, 165, 156, 179, 115, 60, 222, 93]


### ATTACK_CODE_CELL

In [241]:
%%file /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/launcher.py

from ctypes import Structure, c_int, CDLL, c_double, POINTER, Union
from database_utils import Database, Trace, KEY_LEN
from timeit import default_timer as timer
from datetime import timedelta
import pathlib
import struct
import sys
import os

DOUBLE_SIZE = 8

# user controlled parameters
db_file = "traces.db"
new_capture = False
nb_samples = 3000
nb_traces = 50
traces_start_offset = 0
use_all_wave = False

leakage_ranges = [ \
    (1505 , 1510), (1755 , 1765), (2005 , 2015), (2254 , 2278), 
    (1561 , 1565), (1805 , 1820), (2060 , 2075), (2314 , 2326),
    (1615 , 1625), (1865 , 1878), (2120 , 2130), (2370 , 2384), 
    (1670 , 1680), (1920 , 1934), (2175 , 2185), (2425 , 2435)  
]

# store traces on the database
    
if new_capture:
    print("""
    [!] If you are executing this outside of a jupyter notebook, you won't be able to capture new traces unless you setup the chipwhisperer environment.
    [*] If you are inside a jupyter notebook,  please do  the following steps :
     1- Go to the cell "2.0.1  CAPTURE_NEW_TRACES_CELL" :
         1.1- Set your paramaters and make sure the parameter new_traces is set to True
         1.2- Execute the cell 
     2- Go to the cell "2.0.2  ATTACK_CODE_CELL" :
         2.1- Set your parameters and make sure the parameter new_traces is set to False
         2.2- Execute the cell
     3- Go to the cell "2.0.3  ATTACK_LAUNCHER_CELL" :
         3.1- Execute the cell""")
    sys.exit(0)
    
db  = Database(file=db_file)

# perform a query which selects args.number_of_traces traces starting from args.traces_start_offset offset
traces_from_db = Trace.select().limit(nb_traces).offset(traces_start_offset)


# load C lib
try:
    lib = CDLL(pathlib.Path("cpa/cpa_attack.so").absolute().as_posix())
except OSError as err:
    print("Make sure that `make` command was performed and the library was created. \nError : {}".format(err))

# define Ctype structure that matches the Python trace 

class U_WAVE(Union):
    _fields_ = [
        ("intervals", POINTER(c_double) * KEY_LEN),
        ("all", POINTER(c_double)),
    ]      

class TRACE(Structure):
    _fields_ = [
        ("wave", U_WAVE),
        ("textin", c_int * KEY_LEN),
        ("key", c_int * KEY_LEN)
    ]

# defining python pointers to C functions

if use_all_wave :
    intervals = [nb_samples] * KEY_LEN
    c_intervals = (c_int * KEY_LEN)(*intervals) 
    
    init_wave = lib.init_wave                            # python pointer to the C init_wave function 
    init_wave.restype = U_WAVE

    fill_wave = lib.fill_wave                             # python pointer to the C fill_wave function 
    fill_wave.restype = None

else :
    intervals = [ (end - start) for start, end in leakage_ranges]  # calculating the nb_samples for each interval
    c_intervals = (c_int * KEY_LEN)(*intervals)                    # convert it to C int array

    init_intervals = lib.init_intervals                            # python pointer to the C init_intervals function 
    init_intervals.restype = U_WAVE

    fill_intervals = lib.fill_intervals                             # python pointer to the C fill_intervals function 
    fill_intervals.restype = None

# creates a C wave from a binary wave stored in the db 

def handle_wave(nb_samples):
    wave = init_wave(nb_samples)
    wave_bin = struct.unpack_from("d" * nb_samples, trace.wave, 0 * DOUBLE_SIZE)  
    wave_bin = [round(x, 8) for x in wave_bin]
    fill_wave(wave, (c_double * nb_samples)(*wave_bin), nb_samples)
    return wave

def handle_intervals(c_intervals):
    
    wave = init_intervals(c_intervals)                         # allocate memory for all the intervals
    
    for i in range(len(leakage_ranges)):
        start, end = leakage_ranges[i]
        wave_bin = struct.unpack_from("d" * intervals[i], trace.wave, start * DOUBLE_SIZE)  
        wave_bin = [round(x, 8) for x in wave_bin]            # convert the packed data inside the db to samples
        fill_intervals(wave, i, (c_double * intervals[i])(*wave_bin), intervals[i])   # fill the wave structure
    
    return wave

# Create a list of C traces
traces = []

for trace in traces_from_db:                                  # create Traces in C from the Traces in the db
    if use_all_wave :
        wave = handle_wave(nb_samples)                               # create a wave 
    else :
        wave = handle_intervals(c_intervals)                         # create a wave intervals              
    correct_key = list(bytearray.fromhex(trace.key))
    traces.append(TRACE(
        wave,
        (c_int * KEY_LEN)(*list(bytearray.fromhex(trace.textin))),   # convert from Python Array to C Array
        (c_int * KEY_LEN)(*list(bytearray.fromhex(trace.key))),      # convert from Python Array to C Array
    ))
        
traces = (TRACE * nb_traces)(*traces)

# call the function that performs the attack

if use_all_wave :
    start = timer()
    lib.cpa_attack(traces, nb_traces, c_intervals, 1)
    #lib.dpa_attack(traces, nb_traces)
    end   = timer()
else :
    start = timer()
    lib.cpa_attack(traces, nb_traces, c_intervals, 0)
    end   = timer()

for t in traces :                           # free our allocated memroy
    if use_all_wave :
        lib.free_wave(t.wave)
    else :
        lib.free_intervals(t.wave)
        
print("Correct   key: ", end='')
for i in range(KEY_LEN): print(correct_key[i],end=' ')

print("\nThe attack took:  {}".format(timedelta(seconds=end-start)))

Overwriting /home/vagrant/work/projects/chipwhisperer/TME_SCA/TME4/launcher.py


### ATTACK_LAUNCHER_CELL

In [242]:
! python3 launcher.py

Recovered key: 64 21 79 190 114 152 34 43 114 165 156 179 115 60 222 93 
Correct   key: 64 21 79 190 114 152 34 43 114 165 156 179 115 60 222 93 
The attack took:  0:00:00.081424
